In [188]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='west0_pcs7', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39651)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39651)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

Spark master launched!
Creating new Spark session, name: west0_pcs7...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:37


In [187]:
sphynx.stop()

# PC

In [3]:
start_date = "2022-09-09"
end_date = "2022-10-07"

In [5]:
# user 지역
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')
pc_user = load_data_mart("pc", end_date, end_date, "user_master")
pc_user = pc_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
pc_user = pc_user.join(meta_region, pc_user.country_new == meta_region.country_code_iso2, "left")

In [102]:
pc_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name").like("%pcs7%"))

In [15]:
pc_gcoin.select(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+--------+---------+
|paid_use|total_use|
+--------+---------+
|86418100|127208930|
+--------+---------+



In [31]:
pc_gcoin.withColumn("paid_pu", when(col("paid_use") > 0, col("account_id")).otherwise(None)) \
    .join(pc_user.withColumnRenamed("accountid", "account_id").select("account_id", "pubg_region"), "account_id") \
    .groupBy("pubg_region").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), countDistinct("paid_pu").alias("paid_pu"), countDistinct("account_id").alias("total_pu")).toPandas()

,pubg_region,paid_use,total_use,paid_pu,total_pu
0,CN,57594580,84239950,39499,54299
1,NA,2503370,3537840,1648,2229
2,SA,939400,2130640,817,1668
3,KR,15118130,20269900,9970,13028
4,SEA,1803460,3019410,1320,2120
5,JP,299530,426510,186,252
6,TW/HK,616300,856170,389,502
7,CIS,2236300,4147940,1822,3428
8,EMEA,4797060,7876160,3474,5572
9,OC,509970,704410,342,453


In [90]:
pc_gcoin.groupBy("product_name").agg(sum("qty").alias("unit_sold"), sum("paid_use").alias("paid_use"), sum(col("free_use") + col("paid_use")).alias("total_use")).orderBy("unit_sold", ascending=False).toPandas()

,product_name,unit_sold,paid_use,total_use
0,PCS7 GEAR SET,28074,14568420,27793260
1,PCS7 MEGA BUNDLE,17750,44657140,52717500
2,PCS7 CLOTHING BUNDLE,15424,7546665,15269760
3,PCS7 EMOTE PACK,14329,3490685,7164500
4,PCS7 MK12 PACK,9437,6201640,9342630
5,PCS7 PARACHUTE PACK,4350,2465745,4306500
6,PCS7 WEAPON + PARACHUTE SET,3882,6000320,7686360
7,PCS7 SKS PACK,2958,1487485,2928420


In [9]:
pc_user.where((col("lastlogindate") >= start_date) & (col("server_type") == "LIVE")).select(countDistinct("accountid").alias('au')).show()

+--------+
|      au|
+--------+
|10152851|
+--------+



In [8]:
pc_user.where((col("lastlogindate") >= start_date) & (col("server_type") == "LIVE")).groupBy("pubg_region").agg(countDistinct("accountid").alias("au")).toPandas()

,pubg_region,au
0,CN,6389176
1,NA,261771
2,SA,241866
3,KR,1163068
4,SEA,463805
5,Undefined,981
6,JP,35107
7,TW/HK,58333
8,CIS,724853
9,EMEA,773165


### PCS 6

In [180]:
pcs5_gcoin = load_data_mart('pc', "2021-09-08", "2021-09-30", "gcoin_use").where(col("event_name").like("%pcs5%"))
pcs5_user = pcs5_gcoin.select("account_id").distinct()

In [181]:
pcs6_gcoin = load_data_mart('pc', "2022-03-16", "2022-05-06", "gcoin_use").where(col("event_name").like("%pcs6%"))
pcs6_user = pcs6_gcoin.select("account_id").distinct()

In [183]:
pcs5_user.select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    187897|
+--------------------------+



In [182]:
pcs5_user.join(pcs6_user, "account_id").select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                     21678|
+--------------------------+



In [184]:
21678/187897*100

11.53717196123408

In [38]:
load_data_mart('pc', "2022-03-16", "2022-05-06", "gcoin_use").join(pcs6_user, "account_id") \
    .groupBy("event_type").agg(countDistinct("account_id").alias("user_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_user")).orderBy("user_cnt", ascending=False).toPandas()

,event_type,user_cnt,paid_use,total_user
0,esports,154418,140671856,235506290
1,survivorpass,72737,107057209,149906210
2,wsus,70445,706557260,817178000
3,other,57365,53973375,78008300
4,yourshop,54463,33918194,51186740
5,collaboration,46416,118554670,142136790
6,streamer,35877,91003120,106209260
7,holiday,27903,18398410,27197600
8,other,8996,4373865,6559440


In [36]:
pcs6_user_buy_in_pcs6_date = load_data_mart('pc', "2022-03-16", "2022-05-06", "gcoin_use").join(pcs6_user, "account_id") \
    .groupBy("event_name").agg(countDistinct("account_id").alias("user_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_user")).orderBy("user_cnt", ascending=False).toPandas()

In [37]:
pcs6_user_buy_in_pcs6_date

,event_name,user_cnt,paid_use,total_user
0,202203_pcs6,135377,134667356,220988790
1,survivorpass_punkwave,72737,107057209,149906210
2,yourshop5,54463,33918194,51186740
3,202203_wsus,45935,195209300,233843600
4,202204_wsus_progressive,36132,262982725,299082000
5,202203_pcs6_emote,29035,6004500,14517500
6,202203_nier,28390,100341780,114559650
7,202204_rash,26036,65749745,74308860
8,202202_wsus_progressive,19743,134432555,153265000
9,202202_whiteday,16744,13126250,18276480


In [96]:
gcoin = load_data_mart("pc", "2022-09-09", "2022-10-07", "gcoin_use")
pcs6_user_buy_in_pcs7_date = gcoin.join(pcs6_user, "account_id") \
    .groupBy("event_name").agg(countDistinct("account_id").alias("user_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_user")).orderBy("user_cnt", ascending=False).toPandas()

In [101]:
gcoin.join(pcs6_user, "account_id").withColumn("event_type2", trim(col("event_type"))) \
    .groupBy("event_type2").agg(countDistinct("account_id").alias("user_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_user")).orderBy("user_cnt", ascending=False).toPandas()

,event_type2,user_cnt,paid_use,total_user
0,season_workshop,25066,209751580,238017150
1,esports,15481,25392460,32219520
2,other,14980,11468545,15651260
3,yourshop,10713,5535870,7634010
4,wsus,10643,74769913,86132400
5,workshop,10223,19338640,23994970
6,other,7431,10982155,13452520
7,collaboration,6097,6335460,7711060
8,streamer,2481,2304260,2914190
9,streamer,380,299420,394850


In [103]:
gcoin.join(pcs6_user, "account_id").where(col("event_type").like("%streamer%")).select(countDistinct("account_id").alias('user_cnt'), sum("paid_use").alias("paid_use"), sum(col("free_use") + col("paid_use")).alias("total_use")).show(truncate=False)

+--------+--------+---------+
|user_cnt|paid_use|total_use|
+--------+--------+---------+
|2734    |2603680 |3309040  |
+--------+--------+---------+



In [35]:
pcs6_user_buy_in_pcs7_date

,event_name,user_cnt,paid_use,total_user
0,202209_season_workshop,25066,209751580,238017150
1,202209_pcs7,12694,21289165,26568630
2,yourshop8,10713,5535870,7634010
3,202207_workshop,10223,19338640,23994970
4,202209_desertfestival_emote,9616,5299505,7418200
5,202208_thanksgiving,6552,10433595,12659520
6,202209_wsus,5702,23977508,27956600
7,202209_heymama_emote,4887,4868290,5864400
8,202209_desertfestival,4778,3101090,4077690
9,202208_wsus_progressive,4712,28521145,32660800


## EsportsPoint

In [193]:
start_date = "2022-09-09"
end_date = "2022-10-07"

device = 'pc'
env = 'live'
log_name = "PickStatusChanged"

pcs7_pickem = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [190]:
ep_7 = load_schema.lobby(spark, "pc", "live", "PurchaseResult", "2022-09-09", "2022-10-18").where(col("Currency") == "esportspoint")

In [192]:
ep_7.groupBy("ProductId").agg(countDistinct("AccountId").alias("pu")).orderBy("pu", ascending=False).toPandas()

,ProductId,pu
0,itemdesc.45000003,295407
1,itemdesc.17210049,201351
2,itemdesc.17230026,176033
3,itemdesc.12022025,25798
4,itemdesc.12010646,16483


In [194]:
pcs7_pickem.select("AccountId").distinct().join(ep_7.select("AccountId").distinct(), "AccountId").select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   379976|
+-------------------------+



In [198]:
pcs7_ep = load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2022-09-09", "2022-10-07").where((col("Currency").like("%esportspoint%")) & (col("Amount") > 0))

In [199]:
pcs7_ep.groupBy("AccountId").agg(sum("Amount").alias("amount")).where(col("amount") >= 150000).select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                    14829|
+-------------------------+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 42266)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

### PCS6 EsportsPoint

In [195]:
pcs6_pickem = load_schema.lobby(spark, "pc", "live", "PickStatusChanged", "2022-03-16", "2022-05-25").select("AccountId").distinct()

In [196]:
ep_6 = load_schema.lobby(spark, "pc", "live", "PurchaseResult", "2022-03-16", "2022-05-25").where(col("Currency") == "esportspoint")

In [197]:
pcs6_pickem.join(ep_6.select("AccountId").distinct(), "AccountId").select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   226925|
+-------------------------+



In [111]:
ep_6.groupBy("ProductId").agg(countDistinct("AccountId").alias("pu")).orderBy("pu", ascending=False).show(truncate=False)

+-----------------+------+
|ProductId        |pu    |
+-----------------+------+
|itemdesc.45000003|300569|
|itemdesc.17230024|273125|
|itemdesc.12010555|218556|
|itemdesc.12020037|193502|
|itemdesc.17210039|180137|
|itemdesc.12010554|173994|
+-----------------+------+



In [115]:
m416_user = ep_6.where(col("ProductId") == "itemdesc.12010555").select("AccountId").distinct()

In [173]:
pcs6_ep = load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2022-03-16", "2022-05-25").where((col("Currency").like("%esportspoint%")) & (col("Amount") > 0))

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35073)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:35073)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [114]:
pcs6_ep.select("Currency").distinct().show(truncate=False)

+-------------------------+
|Currency                 |
+-------------------------+
|currencydesc.esportspoint|
+-------------------------+



ep 125000넘은 사람들의 m416 스킨 구매율

In [174]:
ep_over = pcs6_ep.groupBy("AccountId").agg(sum("Amount").alias("amount")).where(col("amount") >= 125000)

In [120]:
ep_over.select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   240937|
+-------------------------+



In [121]:
# ep_over 중 ep 사용하여 구매한 유저수
ep_over.join(ep_6, "AccountId").select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   231566|
+-------------------------+



In [175]:
ep_over.join(ep_6, "AccountId").groupBy("ProductId").agg(countDistinct("AccountId").alias("pu")).toPandas()

,ProductId,pu
0,itemdesc.45000003,204302
1,itemdesc.12010554,173994
2,itemdesc.17210039,173867
3,itemdesc.17230024,193549
4,itemdesc.12020037,183229
5,itemdesc.12010555,218549


In [122]:
# ep_over 중 m416 스킨 구매한 유저수
ep_over.join(m416_user, "AccountId").select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   218549|
+-------------------------+



In [123]:
pcs5_ep_purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", "2021-09-08", "2021-10-20").where(col("Currency") == "esportspoint")
pcs5_ep_purchase.groupBy("ProductId").agg(countDistinct("AccountId").alias("pu")).orderBy("pu", ascending=False).toPandas()

,ProductId,pu
0,itemdesc.45000003,200783
1,itemdesc.17230021,164203
2,itemdesc.12020034,52167
3,itemdesc.11070083,18299
4,itemdesc.12031092,15351
5,itemdesc.12010469,11464


## Pickem|

In [144]:
start_date = "2022-09-09"
end_date = "2022-10-07"

device = 'pc'
env = 'live'
log_name = "PickStatusChanged"

pickem = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [11]:
pickem.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- GameId: string (nullable = true)
 |-- PickCount: long (nullable = true)
 |-- PickId: string (nullable = true)
 |-- PickTime: string (nullable = true)
 |-- Reason: string (nullable = true)
 |-- SeasonId: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Type: string (nullable = true)



In [13]:
pickem.select("GameId").distinct().show(truncate=False)

+-------------------------------+
|GameId                         |
+-------------------------------+
|pickem202209.Game.BONUS.AM.A1  |
|pickem202209.Game.BONUS.AM.A4  |
|pickem202209.Game.BONUS.EU.A2  |
|pickem202209.Game.EU.BR        |
|pickem202209.Game.ASIA.BR      |
|pickem202209.Game.BONUS.ASIA.A3|
|pickem202209.Game.BONUS.EU.A3  |
|pickem202209.Game.BONUS.ASIA.A2|
|pickem202209.Game.BONUS.ASIA.A1|
|pickem202209.Game.BONUS.APAC.A2|
|pickem202209.Game.BONUS.EU.A1  |
|pickem202209.Game.APAC.BR      |
|pickem202209.Game.BONUS.AM.A2  |
|pickem202209.Game.BONUS.ASIA.A4|
|pickem202209.Game.BONUS.EU.A4  |
|pickem202209.Game.BONUS.APAC.A3|
|pickem202209.Game.BONUS.AM.A3  |
|pickem202209.Game.BONUS.APAC.A4|
|pickem202209.Game.BONUS.APAC.A1|
|pickem202209.Game.AM.BR        |
+-------------------------------+



In [14]:
pickem.withColumn("br_bonus", when(col("GameId").like("%BONUS%"), lit("bonus")).when(col("GameId").like("%BR%"), lit("br")).otherwise(lit("None"))) \
    .groupBy("br_bonus").agg(countDistinct("AccountId").alias("user_cnt")).show(truncate=False)

+--------+--------+
|br_bonus|user_cnt|
+--------+--------+
|br      |1394295 |
|bonus   |508463  |
+--------+--------+



In [16]:
pickem.select(countDistinct("AccountId").alias("total_user_cnt")).show(truncate=False)

+--------------+
|total_user_cnt|
+--------------+
|1457545       |
+--------------+



### PCS6

In [130]:
load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2022-03-16", "2022-05-06").where((col("Currency").like("%votingcoupon%")) & (col("Amount") > 0)) \
    .withColumn("coupon_type", when(col("Currency").like("%votingcoupon.foc%"), lit("foc")).when(col("Currency").like("%votingcoupon.p%"), lit("p")).otherwise(None)) \
    .groupBy("coupon_type").agg(sum("Amount").alias("amount")).show(truncate=False)

+-----------+------+
|coupon_type|amount|
+-----------+------+
|p          |411238|
|foc        |77015 |
+-----------+------+



In [125]:
coupon = load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2022-04-06", "2022-05-06").where((col("Currency").like("%votingcoupon%")) & (col("Reason") == "pickem-pick")) # 2022-05-25

In [127]:
coupon.select("Currency").distinct().show(truncate=False)

+------------------------------------+
|Currency                            |
+------------------------------------+
|currencydesc.pickem.votingcoupon.p  |
|currencydesc.pickem.votingcoupon.foc|
+------------------------------------+



In [128]:
coupon.groupBy("Currency").agg(sum("Amount").alias("amount")).toPandas()

,Currency,amount
0,currencydesc.pickem.votingcoupon.p,-193662
1,currencydesc.pickem.votingcoupon.foc,-61021


## OES

In [145]:
start_date = "2022-09-09"
end_date = "2022-10-07"

device = 'pc'
env = 'live'
log_name = "CurrencyIncreased"

currency_increased = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [146]:
oes_reward = currency_increased.where((col("Reason") == "outgameevent-mission-reward") & (col("Currency") == "currencydesc.pickem.votingcoupon.foc"))

In [44]:
oes_reward.select(countDistinct("AccountId").alias("oes_reward_user_cnt")).show()

+-------------------+
|oes_reward_user_cnt|
+-------------------+
|            2901942|
+-------------------+



In [147]:
oes_reward_time = oes_reward.groupBy("AccountId").agg(min("Time").alias("oes_time"))

In [148]:
pickem_pick = currency_increased.where((col("Reason") == "pickem-pick") & (col("Currency") == "currencydesc.pickem.votingcoupon.foc"))

In [149]:
pickem_pick_time = pickem_pick.groupBy("AccountId").agg(min("Time").alias("pick_time"))

In [150]:
pcs7_buy_time = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name").like("%pcs7%")).groupBy("account_id").agg(min("Time").alias("buy_time")).withColumnRenamed("account_id", "AccountId")

In [69]:
oes_reward_time.join(pickem_pick_time, "AccountId").select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                  1385715|
+-------------------------+



In [72]:
oes_reward_time.join(pickem_pick_time, "AccountId").join(pcs7_buy_time, "AccountId").where((col("oes_time") < col("pick_time")) & (col("pick_time") < col("buy_time"))).select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                    26917|
+-------------------------+



In [82]:
oes_reward_time.join(pickem_pick_time, "AccountId").join(pcs7_buy_time, "AccountId") \
    .where((col("oes_time") < col("pick_time")) & (col("pick_time") < col("buy_time"))) \
    .join(pcs6_user.withColumnRenamed("account_id", "AccountId").select("AccountId"), "AccountId", "leftanti").select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                    22657|
+-------------------------+



In [139]:
# oes로 투표권 획득하여 투표했지만 PCS7 상품은 구매하지 않은 유저들
oes_tmp = oes_reward_time.join(pickem_pick_time, "AccountId") \
    .join(pcs7_buy_time, "AccountId", "leftanti")
oes_tmp.select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                  1328965|
+-------------------------+



In [140]:
# 중 BONUS 투표에도 참여한 유저 수 
oes_tmp.join(pickem.where(col("GameId").like("%BONUS%")).select("AccountId").distinct(), "AccountId") \
    .select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   407806|
+-------------------------+



In [159]:
heimdall = load_schema.heimdall(spark, "prod", "pc-prod", "2022-09-09", "2022-10-07")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34255)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34255)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [161]:
click_teamfaceoff = heimdall.where((col("category") == "esports_pickem") & (col("action") == "click_tab") & (col("label") == "teamfaceoff")).select("user_id").distinct().withColumnRenamed("user_id")

In [164]:
# OES로 획득한 무료 투표권 사용하여 투표하였고, team faceoff (Bonus투표) 페이지도 진입한 유저 수
oes_reward_time.join(pickem_pick_time, "AccountId") \
    .join(click_teamfaceoff, "AccountId") \
    .select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   939741|
+-------------------------+



In [163]:
# OES로 획득한 무료 투표권 사용하여 투표하였고, team faceoff (Bonus투표) 페이지도 진입하였지만, bonus투표는 하지 않은 유저
oes_reward_time.join(pickem_pick_time, "AccountId") \
    .join(click_teamfaceoff, "AccountId") \
    .join(pickem.where(col("GameId").like("%BONUS%")).select("AccountId").distinct(), "AccountId", "leftanti") \
    .select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   498375|
+-------------------------+



In [152]:
start_date = "2022-04-06"
end_date = "2022-05-06"

device = 'pc'
env = 'live'
log_name = "PickStatusChanged"

pickem_pcs6 = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [153]:
# OES로 획득한 무료 투표권 사용하여 투표하였고, Bonus도 투표한 유저중 pcs6 bonus 투표에도 참여한 유저 수
oes_tmp.join(pickem.where(col("GameId").like("%BONUS%")).select("AccountId").distinct(), "AccountId") \
    .join(pickem_pcs6.where(col("GameId").like("%BONUS%")).select("AccountId").distinct(), "AccountId") \
    .select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                   159036|
+-------------------------+



In [154]:
407806 - 159036

248770

In [167]:
delete_query = """
    INSERT INTO metainfo.meta_esports_pickem 
    VALUES ("PGC 2022", "PC", "2022-10-26", "2022-12-07", "2022-10-26", "2022-11-18", "2022-10-26", "2022-12-07", "202210_pgc"),
    ("PGC 2022", "Console", "2022-10-26", "2022-12-07", "2022-10-26", "2022-11-18", "2022-10-26", "2022-12-07", "202210_pgc");
    """

with mysql.get_connector('pubg_gi') as connector:
        cursor = connector.cursor()
        cursor.execute(delete_query)
        connector.commit()
        print("Finished")

Finished


# Console

In [21]:
start_date = "2022-09-15"
end_date = "2022-10-07"
device = "console"

from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')
console_user = load_data_mart(device, end_date, end_date, "user_master")
console_user = console_user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
console_user = console_user.join(meta_region, console_user.country_new == meta_region.country_code_iso2, "left")

In [88]:
start_date = "2022-09-15"
end_date = "2022-10-07"
gcoin_console = load_data_mart("console", start_date, end_date, "gcoin_use").where(col("event_name").like("%pcs7%"))

In [23]:
gcoin_console.select(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+--------+---------+
|paid_use|total_use|
+--------+---------+
|10797220|14365800 |
+--------+---------+



In [30]:
gcoin_console.withColumn("paid_pu", when(col("paid_use") > 0, col("account_id")).otherwise(None)).select(countDistinct("paid_pu").alias("paid_pu"), countDistinct("account_id").alias("total_pu")).show()

+-------+--------+
|paid_pu|total_pu|
+-------+--------+
|   8141|   10836|
+-------+--------+



In [32]:
gcoin_console.withColumn("paid_pu", when(col("paid_use") > 0, col("account_id")).otherwise(None)) \
    .join(console_user.withColumnRenamed("accountid", "account_id").select("account_id", "pubg_region"), "account_id") \
    .groupBy("pubg_region").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), countDistinct("paid_pu").alias("paid_pu"), countDistinct("account_id").alias("total_pu")).toPandas()

,pubg_region,paid_use,total_use,paid_pu,total_pu
0,CN,78020,93160,44,52
1,NA,6818680,8741700,5290,6777
2,SA,1027970,1504350,690,1079
3,KR,103400,123440,64,71
4,SEA,56520,71870,42,56
5,JP,315200,381260,216,246
6,TW/HK,46980,52550,30,36
7,CIS,88030,362200,52,198
8,EMEA,1843300,2527410,1372,1911
9,OC,419120,507860,341,410


In [91]:
gcoin_console.groupBy("product_name").agg(sum("qty").alias("unit_sold"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).orderBy("unit_sold", ascending=False).toPandas()

,product_name,unit_sold,paid_use,total_use
0,PCS7 Dance - All-Out Brawl,2780,881880,1390000
1,PCS7 CLOTHING BUNDLE,2658,1626910,2631420
2,PCS7 GEAR SET,2157,1620010,2135430
3,Hairstyle 33,1558,532550,779000
4,PCS7 MEGA BUNDLE,1253,3343680,3721410
5,"""PCS7"" UAZ",906,620510,896940
6,PCS7 - Mk12,879,645360,870210
7,PCS7 - Pan,611,483580,604890
8,PCS7 Parachute,562,410120,556380
9,PCS7 Champion Makeup,386,307570,382140


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 36238)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [25]:
console_user.where((col("lastlogindate") >= start_date) & (col("server_type") == "LIVE")).select(countDistinct("accountid").alias('au')).show()

+-------+
|     au|
+-------+
|1265470|
+-------+



In [26]:
console_user.where((col("lastlogindate") >= start_date) & (col("server_type") == "LIVE")).groupBy("pubg_region").agg(countDistinct("accountid").alias("au")).toPandas()

,pubg_region,au
0,CN,18849
1,NA,570276
2,SA,202246
3,KR,6164
4,SEA,22222
5,Undefined,439
6,JP,18791
7,TW/HK,4838
8,CIS,45042
9,EMEA,346450
